In [2]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('../data/merged_metu_ring.csv')
df.head()

,Gender,Residence,Study_Level,Faculty,Grad_School,Visiting_Faculty_GradSchool,Affiliation_Program,Freq_Taxi,Freq_Ring,Freq_Walking,...,Problem_NotClean,Problem_DriverBehavior,Understanding_Routes,Avg_Wait_Time,UseMore_IfFrequent,App_Helping_Ring,MostValuable_Improvement,Overall_Satisfaction,Open_Comments,Department
0,Male,"East Dorms/Guesthouses (1st Dorm, 2nd Dorm, 16...",Undergraduate (Bachelor’s),Faculty of Engineering,NaN,NaN,NaN,Rarely,Sometimes,Always,...,Often,Always,1.0,11–15 minutes,Yes,5.0,Better route coverage,2.0,The buses should have a much better indication...,Electrical and Electronics Engineering
1,Female,"West Dorms/Guesthouses (Isa Demiray Dormitory,...",Undergraduate (Bachelor’s),Faculty of Arts and Sciences,NaN,NaN,NaN,Rarely,Often,Sometimes,...,Sometimes,Never,3.0,11–15 minutes,Yes,5.0,More frequent buses,4.0,NaN,Psychology
2,Female,"West Dorms/Guesthouses (Isa Demiray Dormitory,...",Undergraduate (Bachelor’s),Faculty of Engineering,NaN,NaN,NaN,Rarely,Always,Always,...,Often,Never,5.0,11–15 minutes,Yes,5.0,Better route coverage,3.0,NaN,Computer Engineering
3,Male,"West Dorms/Guesthouses (Isa Demiray Dormitory,...",Undergraduate (Bachelor’s),Faculty of Engineering,NaN,NaN,NaN,Rarely,Always,Always,...,Never,Never,5.0,5–10 minutes,Yes,2.0,Better route coverage,3.0,The weekday evening busses’ route is overlappi...,Civil Engineering
4,Male,Off-campus (within walking distance),Undergraduate (Bachelor’s),Faculty of Economic and Administrative Sciences,NaN,NaN,NaN,Often,Sometimes,Often,...,Never,Never,2.0,11–15 minutes,Yes,4.0,Better real-time tracking,4.0,NaN,International Relations


In [8]:
y_col = "Overall_Satisfaction"

In [9]:
df[y_col].describe()

count    323.000000
mean       2.956656
std        0.840351
min        1.000000
25%        2.000000
50%        3.000000
75%        3.000000
max        5.000000
Name: Overall_Satisfaction, dtype: float64

# Simple Random Sampling

In [11]:
n = len(df)
y = df[y_col].dropna()


In [12]:
y_bar = y.mean()
S2 = y.var(ddof=1)

In [13]:
var_srs = S2 / n
se_srs = np.sqrt(var_srs)

In [14]:
f"popluation_mean: {y_bar}, popluation_variance: {S2}, popluation_std: {np.sqrt(S2)}, popluation_se: {se_srs}"

'popluation_mean: 2.956656346749226, popluation_variance: 0.7061900274984135, popluation_std: 0.8403511334545897, popluation_se: 0.04661429386253546'